In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
import csv
import requests
import json

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

import sqlite3

In [2]:
data = pd.read_json("http://bechdeltest.com/api/v1/getAllMovies", orient="columns")
bechdel_df = pd.DataFrame(data)

In [3]:
bechdel_df.head()

,id,imdbid,rating,title,year
0,8040,0392728,0,Roundhay Garden Scene,1888
1,5433,0000003,0,Pauvre Pierrot,1892
2,5444,0000014,0,Tables Turned on the Gardener,1895
3,6200,0132134,0,"Execution of Mary, Queen of Scots, The",1895
4,6199,0000012,0,"Arrival of a Train, The",1896


In [4]:
bechdel_df.columns = ['id', 'imdbid', 'bechdelRating', 'title', 'year']
bechdel_df.head()

,id,imdbid,bechdelRating,title,year
0,8040,0392728,0,Roundhay Garden Scene,1888
1,5433,0000003,0,Pauvre Pierrot,1892
2,5444,0000014,0,Tables Turned on the Gardener,1895
3,6200,0132134,0,"Execution of Mary, Queen of Scots, The",1895
4,6199,0000012,0,"Arrival of a Train, The",1896


In [5]:
bechdel_df.set_index('id', inplace=True)
bechdel_df.head()

,imdbid,bechdelRating,title,year
id,,,,
8040,0392728,0,Roundhay Garden Scene,1888
5433,0000003,0,Pauvre Pierrot,1892
5444,0000014,0,Tables Turned on the Gardener,1895
6200,0132134,0,"Execution of Mary, Queen of Scots, The",1895
6199,0000012,0,"Arrival of a Train, The",1896


In [6]:
bechdel_df.to_csv("current_bechdel.csv")

In [7]:
# Create the MoviesBechdel class
class Bechdel(Base):
    __tablename__ = 'movies_bechdel', 
    id = Column(Integer, primary_key=True)
    imdbid = Column(Integer)
    bechdelRating = Column(Integer)
    title = Column(String(255))
    year = Column(Integer)

In [8]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///bechdel.sqlite')

In [9]:
# Create the bechdel movies table within the database
Base.metadata.create_all(engine)

In [10]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
conn = sqlite3.connect('bechdel.sqlite')

In [12]:
bechdel_df.to_sql('movies_bechdel', conn, if_exists='append', index=False)

In [13]:
session.commit()

In [14]:
dataRead = pd.read_sql("SELECT * FROM movies_bechdel", conn)

In [15]:
dataRead.head()

,imdbid,bechdelRating,title,year
0,0392728,0,Roundhay Garden Scene,1888
1,0000003,0,Pauvre Pierrot,1892
2,0000014,0,Tables Turned on the Gardener,1895
3,0132134,0,"Execution of Mary, Queen of Scots, The",1895
4,0000012,0,"Arrival of a Train, The",1896
